### All Easy DA - BERT Base Uncased

#### Un-augmented test set
#### Augment only the training set - 7 augmented examples per original

#### Get Original Paper Data

In [1]:
# !pip install sklearn
# !pip install ekphrasis
# !pip install transformers
# !pip install spacy
# !python -m spacy download en_core_web_sm

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds

import transformers

from transformers import BertTokenizer, TFBertModel
from tensorflow.keras import backend as K
from tensorflow import keras

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

import os

import logging
tf.get_logger().setLevel(logging.ERROR)

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
encoder = LabelEncoder()
encoder.classes_ = np.load('../Data/classes.npy', allow_pickle=True)

In [5]:
tf.__version__

'2.6.0'

In [6]:
transformers.__version__

'4.16.2'

In [7]:
# test, train, dev examples from base notebook

train_data_df = pd.read_csv('./Saved_Models/EDA_base_uncased_5aug/All_DA_BERT_base_uncased_train_examples.csv')
dev_data_df = pd.read_csv('./Saved_Models/EDA_base_uncased_5aug/All_DA_BERT_base_uncased_dev_examples.csv')
test_data_df = pd.read_csv('./Saved_Models/EDA_base_uncased_5aug/All_DA_BERT_base_uncased_test_examples.csv')

In [8]:
train_data_df.head()

Unnamed: 0                      post_id  \
0           0                 22448349_gab   
1           1  1178948520201637888_twitter   
2           2                  1482573_gab   
3           3  1097184028149587969_twitter   
4           4  1089569255111176192_twitter   

                                                text  \
0  ['common', 'core', 'weed', 'too', 'much', 'rit...   
1  ['took', 'my', 'nan', 'to', 'the', 'hospital',...   
2  ['<user>', 'well', 'not', 'really', 'islam', '...   
3  ['<user>', 'france', 'in', '<number>', 'after'...   
4  ['i', 'will', 'not', 'tolerate', 'non', 'arab'...   

                    target1    target2            target3  \
0          ['Men', 'Women']  ['Women']           ['None']   
1                  ['None']   ['None']           ['None']   
2                 ['Islam']  ['Other']          ['Islam']   
3        ['Islam', 'Other']  ['Islam']          ['Islam']   
4  ['Arab', 'Men', 'Women']   ['Arab']  ['Arab', 'Islam']   

                                          rationales final_label  \
0                                                 []      normal   
1                                                 []      normal   
2  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,...   offensive   
3                                                 []      normal   
4  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,...  hatespeech   

                                       text_combined  
0  common core weed too much ritalan chem trails ...  
1  took my nan to the hospital for a x ray i turn...  
2  <user> well not really islam does not care for...  
3  <user> france in <number> after muslims take o...  
4  i will not tolerate non arab women slandering ...

In [9]:
X_train_id = train_data_df['post_id']
X_test_id = test_data_df['post_id']
X_dev_id = dev_data_df['post_id']

In [10]:
y_train = train_data_df['final_label']
y_test = test_data_df['final_label']
y_dev = dev_data_df['final_label']

In [11]:
x_train_df = pd.DataFrame({'post_id' : X_train_id.to_list()})
x_dev_df = pd.DataFrame({'post_id' : X_dev_id.to_list()})
x_test_df = pd.DataFrame({'post_id' : X_test_id.to_list()})

# X_train_df = pd.merge(x_train_df, raw_data_final, how='inner', on='post_id')
# X_dev_df = pd.merge(x_dev_df, raw_data_final, how='inner', on='post_id')
# X_test_df = pd.merge(x_test_df, raw_data_final, how='inner', on='post_id')

X_train_text = train_data_df['text_combined'].to_list()
X_dev_text= dev_data_df['text_combined'].to_list()
X_test_text = test_data_df['text_combined'].to_list()

print(len(X_train_text))
print(len(X_dev_text))
print(len(X_test_text))

15383
1923
1923


### Get Augmented Data

In [12]:
# load augmented datasets generated by EDA
# sr = synonym replacement
# ri = random synonym insertion
# rs = random swap
# rd = random deletion
# dataframe name format: method_number 

sr_1_df = pd.read_csv('../test_data_set/EDA_7_0_7_sr_rest_0_1.csv')
ri_1_df = pd.read_csv('../test_data_set/EDA_7_0_7_ri_rest_0_1.csv')
rs_1_df = pd.read_csv('../test_data_set/EDA_7_0_7_rs_rest_0_1.csv')
rd_1_df = pd.read_csv('../test_data_set/EDA_7_0_7_rd_rest_0_1.csv')
all_1_df = pd.read_csv('../test_data_set/EDA_7_all_0_1s.csv')
all_5_df = pd.read_csv('../test_data_set/EDA_7_all_0_5s.csv')

# remove undecided labeled examples
sr_1_df_filtered = sr_1_df[sr_1_df['final_label'] != 'undecided']
ri_1_df_filtered = ri_1_df[ri_1_df['final_label'] != 'undecided']
rs_1_df_filtered = rs_1_df[rs_1_df['final_label'] != 'undecided']
rd_1_df_filtered = rd_1_df[rd_1_df['final_label'] != 'undecided']
all_1_df_filtered = all_1_df[all_1_df['final_label'] != 'undecided']
all_5_df_filtered = all_5_df[all_5_df['final_label'] != 'undecided']

len(sr_1_df_filtered)

153832

In [13]:
# separate train, dev, test for each set
sr_1_df_train = sr_1_df_filtered[sr_1_df_filtered['post_id'].isin(X_train_id)]
ri_1_df_train = ri_1_df_filtered[ri_1_df_filtered['post_id'].isin(X_train_id)]
rs_1_df_train = rs_1_df_filtered[rs_1_df_filtered['post_id'].isin(X_train_id)]
rd_1_df_train = rd_1_df_filtered[rd_1_df_filtered['post_id'].isin(X_train_id)]
all_1_df_train = all_1_df_filtered[all_1_df_filtered['post_id'].isin(X_train_id)]
all_5_df_train = all_5_df_filtered[all_5_df_filtered['post_id'].isin(X_train_id)]

# select text sets

aug_sr_text = sr_1_df_train['text_str'].to_list()
aug_ri_text = ri_1_df_train['text_str'].to_list()
aug_rs_text = rs_1_df_train['text_str'].to_list()
aug_rd_text = rd_1_df_train['text_str'].to_list()
aug_all_1_text = all_1_df_train['text_str'].to_list()
aug_all_5_text = all_5_df_train['text_str'].to_list()

# select label sets

aug_sr_labels = sr_1_df_train['final_label']
aug_ri_labels = ri_1_df_train['final_label']
aug_rs_labels = rs_1_df_train['final_label']
aug_rd_labels = rd_1_df_train['final_label']
aug_all_1_labels = all_1_df_train['final_label']
aug_all_5_labels = all_5_df_train['final_label']

len(aug_sr_text)

123064

#### Convert labels to one-hot encoding

In [14]:
# convert class label to 1 hot encoding

def convert_to_oh(S):
    '''takes a pandas series of text labels and returns one hot encoding equivalent
    0 = normal, 1 = offensive, 2 = hatespeech
    ''' 
    S_numerical = S.apply(lambda x: 0 if x=='normal' else (1 if x=='offensive' else 2))
    S_oh = keras.utils.to_categorical(S_numerical, num_classes = 3, dtype = 'float32')
    return S_oh
    
# original dataset - train, dev, and train
y_train_orig = convert_to_oh(pd.Series(y_train))
y_dev_orig = convert_to_oh(pd.Series(y_dev))
y_test_orig = convert_to_oh(pd.Series(y_test))

# augmented with sr = 0.1
y_train_aug_sr = convert_to_oh(aug_sr_labels)

# augmented with ri = 0.1
y_train_aug_ri = convert_to_oh(aug_ri_labels)

# augmented with rs = 0.1
y_train_aug_rs = convert_to_oh(aug_rs_labels)

# augmented with rd = 0.1
y_train_aug_rd = convert_to_oh(aug_rd_labels)

# augmented with all = 0.1
y_train_all_1 = convert_to_oh(aug_all_1_labels)

# augmented with all = 0.5
y_train_all_5 = convert_to_oh(aug_all_5_labels)


In [15]:
len(aug_ri_text)

123064

In [16]:
len(y_train_aug_ri)

123064

In [17]:
y_train_aug_ri

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

### BERT Model

In [18]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#bert_model = TFBertModel.from_pretrained('bert-base-uncased')

In [19]:
aug_sr_text

['i dont retrieve im bugger off my cosset them edward white he has ii edward white joule and nike not still tinct',
 'i dont think im getting my baby patrick white them white he has two white j and nikes not even touched',
 'i dont suppose im beget my pamper them whiten he has deuce whiten watt second and nike not eve meet',
 'i dont think im getting my baby them white he has two white j atomic number and nikes not even touched',
 'i dont im getting my baby them white he has two j and nikes not even touched',
 'touched dont think im getting my baby them white he has two white j and nikes not even i',
 'i think im getting my baby them white he has two white j and nikes not even touched',
 'i dont think im getting my baby them white he has two white j and nikes not even touched',
 'we figure cannot continue calling ourselves feminists if the rights of all womxn arent addressed yes grammatical gender to a sexual offences public list but meeting place will a trans lesbian bisexual and quee

In [20]:
max_length = 128

def bert_tokenize(train_set, dev_set, test_set, max_length):
    
    train = tokenizer(train_set, max_length=max_length, truncation=True, padding='max_length', return_tensors='tf')
    dev = tokenizer(dev_set, max_length=max_length, truncation=True, padding='max_length', return_tensors='tf')
    test = tokenizer(test_set, max_length=max_length, truncation=True, padding='max_length', return_tensors='tf')
    
    return train, dev, test

X_train_orig, X_dev_orig, X_test_orig = bert_tokenize(X_train_text, X_dev_text, X_test_text, max_length)

X_train_aug_sr, X_dev_aug_sr, X_test_aug_sr = bert_tokenize(aug_sr_text, X_dev_text, X_test_text, max_length)

X_train_aug_ri, X_dev_aug_ri, X_test_aug_ri = bert_tokenize(aug_ri_text, X_dev_text, X_test_text, max_length)

X_train_aug_rs, X_dev_aug_rs, X_test_aug_rs = bert_tokenize(aug_rs_text, X_dev_text, X_test_text, max_length)

X_train_aug_rd, X_dev_aug_rd, X_test_aug_rd = bert_tokenize(aug_rd_text, X_dev_text, X_test_text, max_length)

X_train_all_1, X_dev_all_1, X_test_all_1 = bert_tokenize(aug_all_1_text, X_dev_text, X_test_text, max_length)

X_train_all_5, X_dev_all_5, X_test_all_5 = bert_tokenize(aug_all_5_text, X_dev_text, X_test_text, max_length)


In [21]:
#tokenizer.save_pretrained("./Tokenizer_ALL_EDA_BERT_base_uncased")

In [22]:
X_train_orig.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [23]:
X_train_aug_sr.input_ids

<tf.Tensor: shape=(123064, 128), dtype=int32, numpy=
array([[  101,  1045,  2123, ...,     0,     0,     0],
       [  101,  1045,  2123, ...,     0,     0,     0],
       [  101,  1045,  2123, ...,     0,     0,     0],
       ...,
       [  101,  1996, 18414, ...,     0,     0,     0],
       [  101,  1996,  3644, ...,     0,     0,     0],
       [  101,  1996,  3644, ...,     0,     0,     0]])>

In [24]:
X_train_aug_ri.token_type_ids

<tf.Tensor: shape=(123064, 128), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>

In [25]:
X_train_all_1.attention_mask

<tf.Tensor: shape=(123064, 128), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>

In [26]:
X_train_all_5.input_ids

<tf.Tensor: shape=(123064, 128), dtype=int32, numpy=
array([[  101,  2317,  2317, ...,     0,     0,     0],
       [  101,  3944,  1045, ...,     0,     0,     0],
       [  101,  1046,  2317, ...,     0,     0,     0],
       ...,
       [  101,  9467,  3993, ...,     0,     0,     0],
       [  101,  1996, 18414, ...,     0,     0,     0],
       [  101,  1996,  3644, ...,     0,     0,     0]])>

In [27]:
from keras import backend as K

def balanced_recall(y_true, y_pred):
    """This function calculates the balanced recall metric
    recall = TP / (TP + FN)
    """
    recall_by_class = 0
    # iterate over each predicted class to get class-specific metric
    for i in range(y_pred.shape[1]):
        y_pred_class = y_pred[:, i]
        y_true_class = y_true[:, i]
        true_positives = K.sum(K.round(K.clip(y_true_class * y_pred_class, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true_class, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        recall_by_class = recall_by_class + recall
    return recall_by_class / y_pred.shape[1]

def balanced_precision(y_true, y_pred):
    """This function calculates the balanced precision metric
    precision = TP / (TP + FP)
    """
    precision_by_class = 0
    # iterate over each predicted class to get class-specific metric
    for i in range(y_pred.shape[1]):
        y_pred_class = y_pred[:, i]
        y_true_class = y_true[:, i]
        true_positives = K.sum(K.round(K.clip(y_true_class * y_pred_class, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred_class, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        precision_by_class = precision_by_class + precision
    # return average balanced metric for each class
    return precision_by_class / y_pred.shape[1]

def balanced_f1_score(y_true, y_pred):
    """This function calculates the F1 score metric"""
    precision = balanced_precision(y_true, y_pred)
    recall = balanced_recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [28]:
def create_classification_model(bert_model, hidden_size = 5, 
                                train_layers = -1, 
                                optimizer=tf.keras.optimizers.Adam()):
    """
    Build a simple classification model with BERT. Let's keep it simple and don't add dropout, layer norms, etc.
    """

    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                  'token_type_ids': token_type_ids,
                  'attention_mask': attention_mask}


    #restrict training to the train_layers outer transformer layers
    if not train_layers == -1:

            retrain_layers = []

            for retrain_layer_number in range(train_layers):

                layer_code = '_' + str(11 - retrain_layer_number)
                retrain_layers.append(layer_code)

            for w in bert_model.weights:
                if not any([x in w.name for x in retrain_layers]):
                    w._trainable = False


    bert_out = bert_model(bert_inputs)
    
    net = bert_out[0]
    
    classification_token = tf.keras.layers.Lambda(lambda x: x[:,0,:], name='get_first_vector')(net)
    
    dropout1 = tf.keras.layers.Dropout(0.4, name="dropout1")(classification_token)
    
    hidden = tf.keras.layers.Dense(hidden_size, name='hidden_layer')(dropout1)
    
    dropout2 = tf.keras.layers.Dropout(0.4, name="dropout2")(hidden)

    classification = tf.keras.layers.Dense(3, activation='sigmoid',name='classification_layer')(dropout2)

    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], 
                                          outputs=[classification])
    
    METRICS = [tf.keras.metrics.CategoricalAccuracy(name="accuracy"), 
               balanced_recall, 
               balanced_precision, 
               balanced_f1_score,
               tf.keras.metrics.AUC(curve='ROC', name="auc_roc")]
    
    
    classification_model.compile(optimizer=optimizer,
                            loss=tf.keras.losses.CategoricalCrossentropy(),
                            metrics= METRICS)


    return classification_model




#     classification_model.compile(optimizer=optimizer,
#                             loss=tf.keras.losses.CategoricalCrossentropy(),
#                             metrics=tf.keras.metrics.CategoricalAccuracy('accuracy'))

In [29]:
def fine_tune_BERT(x_train, x_dev, x_test, y_train, y_dev, y_test, name, learning_rate = 5e-05, 
                   epsilon=1e-08, train_layers = -1, epochs = 10, batch_size = 16):
    ''' Fine tunes BERT base uncased with given data, allows your to set some hyperparameters
        returns test set accuracy, f1 score, and AUC_ROC score
    '''
    try:
        del classification_model
    except:
        pass

    try:
        del bert_model
    except:
        pass
    
    tf.keras.backend.clear_session()
    bert_model = TFBertModel.from_pretrained('bert-base-uncased')

    # early stopping callback
    
    earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', 
                                                      patience = 4,
                                                      restore_best_weights = True)
    
    # Create a callback that saves the model's weights
    
    path_name = './Saved_Models/EDA_b_7aug/' + name + '/' + name

    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=path_name, 
                                                     save_weights_only=True,
                                                     verbose=1,
                                                     monitor='val_accuracy',
                                                     save_best_only=True)
    
    # create classification model
    classification_model = create_classification_model(bert_model, 
                                                       optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=epsilon),
                                                       train_layers=train_layers)    
    
    model_fit = classification_model.fit([x_train.input_ids, x_train.token_type_ids, x_train.attention_mask],
                         y_train,
                         validation_data=([x_dev.input_ids, x_dev.token_type_ids, x_dev.attention_mask],
                         y_dev),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks = [earlystop_callback, cp_callback])
    
    y_preds_array = classification_model.predict([x_test.input_ids, x_test.token_type_ids, x_test.attention_mask])

    # convert to predicted one-hot encoding

    from keras.utils.np_utils import to_categorical
    y_preds = to_categorical(np.argmax(y_preds_array, 1), dtype = "int64")

    # convert back to labels

    y_test_cat = np.argmax(y_test, axis=1)
    y_preds_cat = np.argmax(y_preds, axis=1)
    
    # calculate metrics
    Accuracy = accuracy_score(y_test_cat, y_preds_cat)

    Macro_F1 = f1_score(y_test_cat, y_preds_cat, average='macro')

    ROC_AUC = roc_auc_score(y_test, y_preds, multi_class='ovo',average='macro')
    
    metrics_history = model_fit.history
    
    return Accuracy, Macro_F1, ROC_AUC, metrics_history

In [38]:
%%time
# original data set
Accuracy_orig, Macro_F1_orig, ROC_AUC_orig, metrics_orig = fine_tune_BERT(X_train_orig, X_dev_orig, X_test_orig, 
                                                            y_train_orig, y_dev_orig, y_test_orig, 'orig_data_7aug',
                                                            learning_rate = 2e-05, epsilon=1e-08, 
                                                            train_layers = 1, epochs = 30, batch_size = 64)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/30
241/241 [==============================] - 61s 212ms/step - loss: 1.2338 - accuracy: 0.4309 - balanced_recall: 0.6037 - balanced_precision: 0.3946 - balanced_f1_score: 0.4770 - auc_roc: 0.6007 - val_loss: 0.9359 - val_accuracy: 0.5413 - val_balanced_recall: 0.7386 - val_balanced_precision: 0.4750 - val_balanced_f1_score: 0.5767 - val_auc_roc: 0.7256

Epoch 00001: val_accuracy improved from -inf to 0.54134, saving model to ./Saved_Models/EDA_b_7aug/orig_data_7aug\orig_data_7aug
Epoch 2/30
241/241 [==============================] - 48s 196ms/step - loss: 1.0920 - accuracy: 0.4888 - balanced_recall: 0.6728 - balanced_precision: 0.4383 - balanced_f1_score: 0.5305 - auc_roc: 0.6679 - val_loss: 0.8822 - val_accuracy: 0.5803 - val_balanced_recall: 0.7674 - val_balanced_precision: 0.5071 - val_balanced_f1_score: 0.6096 - val_auc_roc: 0.7630

Epoch 00002: val_accuracy improved from 0.54134 to 0.58034, saving model to ./Saved_Models/EDA_b_7aug/orig_data_7aug\orig_data_7aug
Epoch 3/30


241/241 [==============================] - 44s 182ms/step - loss: 0.7755 - accuracy: 0.6572 - balanced_recall: 0.8371 - balanced_precision: 0.5315 - balanced_f1_score: 0.6499 - auc_roc: 0.7999 - val_loss: 0.7530 - val_accuracy: 0.6625 - val_balanced_recall: 0.8510 - val_balanced_precision: 0.5259 - val_balanced_f1_score: 0.6486 - val_auc_roc: 0.8186

Epoch 00018: val_accuracy did not improve from 0.67343
Wall time: 15min 30s


In [39]:
%%time
# augmented with sr = 0.1
Accuracy_aug_sr, Macro_F1_aug_sr, ROC_AUC_aug_sr, metrics_sr = fine_tune_BERT(X_train_aug_sr, X_dev_aug_sr, X_test_aug_sr, 
                                                            y_train_aug_sr, y_dev_orig, y_test_orig, 'EDA_sr_7aug', 
                                                            learning_rate = 2e-05, epsilon=1e-08, 
                                                            train_layers = 1, epochs = 30, batch_size = 64)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/30
1923/1923 [==============================] - 337s 171ms/step - loss: 1.0654 - accuracy: 0.5058 - balanced_recall: 0.6817 - balanced_precision: 0.4377 - balanced_f1_score: 0.5327 - auc_roc: 0.6720 - val_loss: 0.7992 - val_accuracy: 0.6526 - val_balanced_recall: 0.8225 - val_balanced_precision: 0.5266 - val_balanced_f1_score: 0.6408 - val_auc_roc: 0.7984

Epoch 00001: val_accuracy improved from -inf to 0.65263, saving model to ./Saved_Models/EDA_b_7aug/EDA_sr_7aug\EDA_sr_7aug
Epoch 2/30
1923/1923 [==============================] - 320s 166ms/step - loss: 0.9039 - accuracy: 0.5821 - balanced_recall: 0.7552 - balanced_precision: 0.4837 - balanced_f1_score: 0.5893 - auc_roc: 0.7412 - val_loss: 0.7635 - val_accuracy: 0.6661 - val_balanced_recall: 0.8243 - val_balanced_precision: 0.5435 - val_balanced_f1_score: 0.6539 - val_auc_roc: 0.8175

Epoch 00002: val_accuracy improved from 0.65263 to 0.66615, saving model to ./Saved_Models/EDA_b_7aug/EDA_sr_7aug\EDA_sr_7aug
Epoch 3/30
1923/1

In [40]:
%%time
# augmented with ri = 0.1
Accuracy_aug_ri, Macro_F1_aug_ri, ROC_AUC_aug_ri, metrics_ri = fine_tune_BERT(X_train_aug_ri, X_dev_aug_ri, X_test_aug_ri, 
                                                            y_train_aug_ri, y_dev_orig, y_test_orig, 'EDA_ri_7aug', 
                                                            learning_rate = 2e-05, epsilon=1e-08, 
                                                            train_layers = 1, epochs = 30, batch_size = 64)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/30
1923/1923 [==============================] - 331s 168ms/step - loss: 1.1106 - accuracy: 0.4924 - balanced_recall: 0.6263 - balanced_precision: 0.4299 - balanced_f1_score: 0.5094 - auc_roc: 0.6563 - val_loss: 0.8174 - val_accuracy: 0.6433 - val_balanced_recall: 0.7299 - val_balanced_precision: 0.5296 - val_balanced_f1_score: 0.6129 - val_auc_roc: 0.7798

Epoch 00001: val_accuracy improved from -inf to 0.64327, saving model to ./Saved_Models/EDA_b_7aug/EDA_ri_7aug\EDA_ri_7aug
Epoch 2/30
1923/1923 [==============================] - 320s 166ms/step - loss: 0.9249 - accuracy: 0.5825 - balanced_recall: 0.6968 - balanced_precision: 0.4871 - balanced_f1_score: 0.5729 - auc_roc: 0.7312 - val_loss: 0.7719 - val_accuracy: 0.6672 - val_balanced_recall: 0.7478 - val_balanced_precision: 0.5352 - val_balanced_f1_score: 0.6235 - val_auc_roc: 0.7973

Epoch 00002: val_accuracy improved from 0.64327 to 0.66719, saving model to ./Saved_Models/EDA_b_7aug/EDA_ri_7aug\EDA_ri_7aug
Epoch 3/30
1923/1

In [41]:
%%time
# augmented with rs = 0.1
Accuracy_aug_rs, Macro_F1_aug_rs, ROC_AUC_aug_rs, metrics_rs = fine_tune_BERT(X_train_aug_rs, X_dev_aug_rs, X_test_aug_rs, 
                                                            y_train_aug_rs, y_dev_orig, y_test_orig, 'EDA_rs_7aug',
                                                            learning_rate = 2e-05, epsilon=1e-08, 
                                                            train_layers = 1, epochs = 30, batch_size = 64)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/30
1923/1923 [==============================] - 331s 168ms/step - loss: 0.9771 - accuracy: 0.5296 - balanced_recall: 0.6080 - balanced_precision: 0.3666 - balanced_f1_score: 0.4559 - auc_roc: 0.5736 - val_loss: 0.7984 - val_accuracy: 0.6422 - val_balanced_recall: 0.6110 - val_balanced_precision: 0.3549 - val_balanced_f1_score: 0.4476 - val_auc_roc: 0.6092

Epoch 00001: val_accuracy improved from -inf to 0.64223, saving model to ./Saved_Models/EDA_b_7aug/EDA_rs_7aug\EDA_rs_7aug
Epoch 2/30
1923/1923 [==============================] - 320s 167ms/step - loss: 0.8616 - accuracy: 0.6068 - balanced_recall: 0.6311 - balanced_precision: 0.3737 - balanced_f1_score: 0.4684 - auc_roc: 0.5876 - val_loss: 0.7601 - val_accuracy: 0.6682 - val_balanced_recall: 0.5978 - val_balanced_precision: 0.3489 - val_balanced_f1_score: 0.4394 - val_auc_roc: 0.6177

Epoch 00002: val_accuracy improved from 0.64223 to 0.66823, saving model to ./Saved_Models/EDA_b_7aug/EDA_rs_7aug\EDA_rs_7aug
Epoch 3/30
1923/1

In [42]:
%%time
# augmented with rd = 0.1
Accuracy_aug_rd, Macro_F1_aug_rd, ROC_AUC_aug_rd, metrics_rd = fine_tune_BERT(X_train_aug_rd, X_dev_aug_rd, X_test_aug_rd, 
                                                            y_train_aug_rd, y_dev_orig, y_test_orig, 'EDA_rd_7aug',
                                                            learning_rate = 2e-05, epsilon=1e-08, 
                                                            train_layers = 1, epochs = 30, batch_size = 64)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/30
1923/1923 [==============================] - 326s 165ms/step - loss: 0.9840 - accuracy: 0.5101 - balanced_recall: 0.6312 - balanced_precision: 0.3999 - balanced_f1_score: 0.4882 - auc_roc: 0.6279 - val_loss: 0.7872 - val_accuracy: 0.6495 - val_balanced_recall: 0.6911 - val_balanced_precision: 0.5134 - val_balanced_f1_score: 0.5882 - val_auc_roc: 0.7540

Epoch 00001: val_accuracy improved from -inf to 0.64951, saving model to ./Saved_Models/EDA_b_7aug/EDA_rd_7aug\EDA_rd_7aug
Epoch 2/30
1923/1923 [==============================] - 316s 164ms/step - loss: 0.8886 - accuracy: 0.5830 - balanced_recall: 0.6143 - balanced_precision: 0.4209 - balanced_f1_score: 0.4980 - auc_roc: 0.6548 - val_loss: 0.7538 - val_accuracy: 0.6771 - val_balanced_recall: 0.6319 - val_balanced_precision: 0.4928 - val_balanced_f1_score: 0.5524 - val_auc_roc: 0.7331

Epoch 00002: val_accuracy improved from 0.64951 to 0.67707, saving model to ./Saved_Models/EDA_b_7aug/EDA_rd_7aug\EDA_rd_7aug
Epoch 3/30
1923/1

In [43]:
%%time
# augmented with all = 0.1
Accuracy_aug_all_1, Macro_F1_aug_all_1, ROC_AUC_aug_all_1, metrics_all_1 = fine_tune_BERT(X_train_all_1, X_dev_all_1, X_test_all_1, 
                                                            y_train_all_1, y_dev_orig, y_test_orig, 'EDA_all_1_7aug',
                                                            learning_rate = 2e-05, epsilon=1e-08, 
                                                            train_layers = 1, epochs = 30, batch_size = 64)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/30
1923/1923 [==============================] - 326s 165ms/step - loss: 1.0363 - accuracy: 0.5265 - balanced_recall: 0.6921 - balanced_precision: 0.4581 - balanced_f1_score: 0.5509 - auc_roc: 0.6998 - val_loss: 0.7945 - val_accuracy: 0.6365 - val_balanced_recall: 0.7896 - val_balanced_precision: 0.5438 - val_balanced_f1_score: 0.6429 - val_auc_roc: 0.8135

Epoch 00001: val_accuracy improved from -inf to 0.63651, saving model to ./Saved_Models/EDA_b_7aug/EDA_all_1_7aug\EDA_all_1_7aug
Epoch 2/30
1923/1923 [==============================] - 318s 165ms/step - loss: 0.8652 - accuracy: 0.6077 - balanced_recall: 0.7669 - balanced_precision: 0.5107 - balanced_f1_score: 0.6127 - auc_roc: 0.7761 - val_loss: 0.7558 - val_accuracy: 0.6719 - val_balanced_recall: 0.8198 - val_balanced_precision: 0.5463 - val_balanced_f1_score: 0.6545 - val_auc_roc: 0.8276

Epoch 00002: val_accuracy improved from 0.63651 to 0.67187, saving model to ./Saved_Models/EDA_b_7aug/EDA_all_1_7aug\EDA_all_1_7aug
Epoch

In [44]:
%%time
# augmented with all = 0.5
Accuracy_aug_all_5, Macro_F1_aug_all_5, ROC_AUC_aug_all_5, metrics_all_5 = fine_tune_BERT(X_train_all_5, X_dev_all_5, X_test_all_5, 
                                                            y_train_all_5, y_dev_orig, y_test_orig, 'EDA_all_5_7aug',
                                                            learning_rate = 2e-05, epsilon=1e-08, 
                                                            train_layers = 1, epochs = 30, batch_size = 64)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/30
1923/1923 [==============================] - 328s 164ms/step - loss: 1.0732 - accuracy: 0.4818 - balanced_recall: 0.6030 - balanced_precision: 0.4094 - balanced_f1_score: 0.4868 - auc_roc: 0.6305 - val_loss: 0.8607 - val_accuracy: 0.6167 - val_balanced_recall: 0.7372 - val_balanced_precision: 0.5039 - val_balanced_f1_score: 0.5974 - val_auc_roc: 0.7549

Epoch 00001: val_accuracy improved from -inf to 0.61674, saving model to ./Saved_Models/EDA_b_7aug/EDA_all_5_7aug\EDA_all_5_7aug
Epoch 2/30
1923/1923 [==============================] - 317s 165ms/step - loss: 0.9501 - accuracy: 0.5557 - balanced_recall: 0.6791 - balanced_precision: 0.4418 - balanced_f1_score: 0.5345 - auc_roc: 0.6882 - val_loss: 0.7975 - val_accuracy: 0.6635 - val_balanced_recall: 0.7958 - val_balanced_precision: 0.5103 - val_balanced_f1_score: 0.6204 - val_auc_roc: 0.7956

Epoch 00002: val_accuracy improved from 0.61674 to 0.66355, saving model to ./Saved_Models/EDA_b_7aug/EDA_all_5_7aug\EDA_all_5_7aug
Epoch

In [45]:
Accuracy_orig, Macro_F1_orig, ROC_AUC_orig, metrics_orig

(0.6895475819032761,
 0.6615269965582669,
 0.7559446883730452,
 {'loss': [1.233755350112915,
   1.0920292139053345,
   1.0057977437973022,
   0.9597842693328857,
   0.926659345626831,
   0.8996855020523071,
   0.8933510780334473,
   0.8742587566375732,
   0.8538693785667419,
   0.8348292112350464,
   0.8340325951576233,
   0.8265409469604492,
   0.814570426940918,
   0.8129034638404846,
   0.7966391444206238,
   0.7905299663543701,
   0.7824797630310059,
   0.7754675149917603],
  'accuracy': [0.4309302568435669,
   0.4887863099575043,
   0.5371513962745667,
   0.5547682642936707,
   0.5778456926345825,
   0.5879217386245728,
   0.594032347202301,
   0.6030033230781555,
   0.6165897250175476,
   0.6240655183792114,
   0.6289410591125488,
   0.6334915161132812,
   0.6343365907669067,
   0.6399921774864197,
   0.6473379731178284,
   0.6480530500411987,
   0.652733564376831,
   0.6572189927101135],
  'balanced_recall': [0.6036643981933594,
   0.6728057265281677,
   0.7158060669898987,
   0

In [46]:
# Accuracy_aug_sr, Macro_F1_aug_sr, ROC_AUC_aug_sr, metrics_sr

In [47]:
# Accuracy_aug_ri, Macro_F1_aug_ri, ROC_AUC_aug_ri, metrics_ri

In [48]:
# Accuracy_aug_rs, Macro_F1_aug_rs, ROC_AUC_aug_rs, metrics_rs

In [49]:
# Accuracy_aug_rd, Macro_F1_aug_rd, ROC_AUC_aug_rd, metrics_rd

In [50]:
# Accuracy_aug_all_1, Macro_F1_aug_all_1, ROC_AUC_aug_all_1, metrics_all_1

In [51]:
# Accuracy_aug_all_5, Macro_F1_aug_all_5, ROC_AUC_aug_all_5, metrics_all_5

In [52]:
trial_name_list = ['Original Data', 'Augmented SR 0.1', 'Augmented RI 0.1', 
                   'Augmented RS 0.1', 'Augmented RD 0.1', 'Augmented All 0.1', 'Augmented All 0.5']

acc_list = [Accuracy_orig, Accuracy_aug_sr, Accuracy_aug_ri, Accuracy_aug_rs, 
            Accuracy_aug_rd, Accuracy_aug_all_1, Accuracy_aug_all_5]

macro_f1_list = [Macro_F1_orig, Macro_F1_aug_sr, Macro_F1_aug_ri, Macro_F1_aug_rs, 
                 Macro_F1_aug_rd, Macro_F1_aug_all_1, Macro_F1_aug_all_5]

roc_auc_list = [ROC_AUC_orig, ROC_AUC_aug_sr, ROC_AUC_aug_ri, ROC_AUC_aug_rs, 
                ROC_AUC_aug_rd, ROC_AUC_aug_all_1, ROC_AUC_aug_all_5]

In [53]:
result_dict = {'Trial Name' : trial_name_list, 'Test Accuracy Score' : acc_list, 
               'Test Macro F1 Score' : macro_f1_list, 'Test ROC AUC Score' : roc_auc_list}

In [54]:
results_df = pd.DataFrame(result_dict)

results_df

Trial Name  Test Accuracy Score  Test Macro F1 Score  \
0      Original Data             0.689548             0.661527   
1   Augmented SR 0.1             0.693188             0.674916   
2   Augmented RI 0.1             0.706708             0.687033   
3   Augmented RS 0.1             0.706708             0.681818   
4   Augmented RD 0.1             0.711908             0.687497   
5  Augmented All 0.1             0.703068             0.688082   
6  Augmented All 0.5             0.706188             0.680916   

   Test ROC AUC Score  
0            0.755945  
1            0.761387  
2            0.770935  
3            0.768938  
4            0.772079  
5            0.771588  
6            0.766816

In [55]:
results_df.to_csv('./Saved_Models/EDA_b_7aug/All_DA_BERT_base_uncased_7aug.csv')

In [56]:
metrics_org_df = pd.DataFrame(metrics_orig)

metrics_org_df

loss  accuracy  balanced_recall  balanced_precision  \
0   1.233755  0.430930         0.603664            0.394637   
1   1.092029  0.488786         0.672806            0.438259   
2   1.005798  0.537151         0.715806            0.466766   
3   0.959784  0.554768         0.746344            0.482317   
4   0.926659  0.577846         0.762145            0.491564   
5   0.899686  0.587922         0.772701            0.497543   
6   0.893351  0.594032         0.777802            0.498901   
7   0.874259  0.603003         0.784147            0.503362   
8   0.853869  0.616590         0.792631            0.508708   
9   0.834829  0.624066         0.804852            0.514851   
10  0.834033  0.628941         0.810027            0.514554   
11  0.826541  0.633492         0.811159            0.517514   
12  0.814570  0.634337         0.818522            0.521090   
13  0.812903  0.639992         0.818035            0.520020   
14  0.796639  0.647338         0.828304            0.525643   
15  0.790530  0.648053         0.827063            0.523829   
16  0.782480  0.652734         0.832407            0.525541   
17  0.775468  0.657219         0.837080            0.531483   

    balanced_f1_score   auc_roc  val_loss  val_accuracy  val_balanced_recall  \
0            0.476969  0.600730  0.935943      0.541342             0.738644   
1            0.530522  0.667857  0.882169      0.580343             0.767425   
2            0.564744  0.707385  0.841519      0.602184             0.827158   
3            0.585639  0.725457  0.822178      0.621945             0.797374   
4            0.597318  0.739321  0.811821      0.616745             0.839402   
5            0.605062  0.750490  0.803874      0.633385             0.810025   
6            0.607629  0.752663  0.784500      0.640146             0.829581   
7            0.612820  0.761772  0.785233      0.643266             0.835567   
8            0.619340  0.769833  0.770479      0.651066             0.844079   
9            0.627667  0.776072  0.771333      0.656266             0.851367   
10           0.629047  0.776655  0.765764      0.657826             0.829321   
11           0.631596  0.781231  0.759917      0.661466             0.841939   
12           0.636481  0.785177  0.760278      0.665107             0.838581   
13           0.635543  0.785647  0.755641      0.673427             0.841984   
14           0.642862  0.792531  0.750046      0.661466             0.844198   
15           0.641087  0.793235  0.753599      0.661986             0.854173   
16           0.643973  0.795278  0.748572      0.669267             0.835336   
17           0.649903  0.799937  0.752971      0.662507             0.850958   

    val_balanced_precision  val_balanced_f1_score  val_auc_roc  
0                 0.474976               0.576728     0.725607  
1                 0.507089               0.609558     0.763008  
2                 0.510350               0.629709     0.776773  
3                 0.520672               0.628722     0.787038  
4                 0.515825               0.637494     0.790285  
5                 0.532117               0.641105     0.799452  
6                 0.520853               0.638520     0.803341  
7                 0.530814               0.647812     0.800886  
8                 0.528028               0.648193     0.805009  
9                 0.518256               0.642759     0.807783  
10                0.526315               0.642604     0.814271  
11                0.526763               0.646653     0.813825  
12                0.523775               0.643353     0.816406  
13                0.523100               0.643847     0.816679  
14                0.533854               0.652741     0.817720  
15                0.520972               0.645689     0.814159  
16                0.533044               0.649443     0.820877  
17                0.525918               0.648612     0.818557

In [57]:
metrics_list = [metrics_orig, metrics_sr, metrics_ri, metrics_rs, metrics_rd, metrics_all_1, metrics_all_5]
name_list = ['fit_metrics_orig.csv', 'fit_metrics_sr.csv', 'fit_metrics_ri.csv', 'fit_metrics_rs.csv', 'fit_metrics_rd.csv', 'fit_metrics_all_1.csv', 'fit_metrics_all_5.csv']

i = 0
for m in metrics_list:
    df = pd.DataFrame(m)
    df.to_csv(name_list[i])
    i += 1

        